# Distributed Data Parallel - Multi GPU

We are performing DDP by distributing the data across multiple GPU. We are using the pretrained ResNet 50 Model. The model is trained on the spectrogram images generated from EEG signal data

In [2]:
import os
import shutil
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.cuda.amp import GradScaler, autocast
from torchvision.models import resnet50
import torch.multiprocessing as mp
import torch.distributed as dist
import torch.optim as optim
import torch.nn.functional as F 
import matplotlib.pyplot as plt
import dask.array as da
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import time

### Data Setup for Model Training

In [3]:
# Split the dataset into train, validation and test
def create_splits(data_dir, output_dir, train_size=0.7, val_size=0.15, test_size=0.15):
    classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    splits = {'train': train_size, 'val': val_size, 'test': test_size}
 
    assert sum(splits.values()) == 1, "Sum of split sizes should be 1."
 
    # Output directories for the splits
    for split in splits.keys():
        for cls in classes:
            os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)
 
    # Iterate over each class directory
    for cls in classes:
        class_dir = os.path.join(data_dir, cls)
        images = [img for img in os.listdir(class_dir) if img.lower().endswith(('png', 'jpg', 'jpeg'))]
        # Stratified splits
        train_val, test = train_test_split(images, test_size=splits['test'], random_state=42, stratify=None)
        train, val = train_test_split(train_val, test_size=splits['val'] / (splits['train'] + splits['val']), random_state=42, stratify=None)
 
        # Function to copy files to directories
        def copy_files(files, split):
            for f in files:
                src = os.path.join(class_dir, f)
                dst = os.path.join(output_dir, split, cls, f)
                shutil.copy(src, dst)
 
        # Copy files to respective split directories
        copy_files(train, 'train')
        copy_files(val, 'val')
        copy_files(test, 'test')
 
 
data_dir = '/kaggle/input/dataset24/dataset_24'
output_dir = '/kaggle/working/dataset_splits/'
create_splits(data_dir, output_dir)

### Data Augmentation using Dask

In [4]:
# Data Augmentation using dask
class CustomDataset(Dataset):
    def __init__(self, ddf, transform=None):
        self.ddf = ddf
        self.transform = transform

    def __len__(self):
        return len(self.ddf)

    def __getitem__(self, idx):
        row = self.ddf.iloc[idx]
        image_path = row['path']
        class_label = row['class']
        transformed_image = row['transformed_image']
        if isinstance(transformed_image, str):
            image = Image.open(transformed_image)
        else:
            image = Image.fromarray(transformed_image)
        if self.transform:
            image = self.transform(image)
        return image, class_label

def add_augmented_images(root_dir, ddf):

    # check for data folder
    os.makedirs(root_dir, exist_ok=True)
    # iterate over dask dataframe containing transformed images
    for _, row in ddf.iterrows():
        class_name = row['class']
        image_path = row['path']
        class_dir = os.path.join(root_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        image_filename = os.path.basename(image_path)
        image_dest_path = os.path.join(class_dir, image_filename)
        if not os.path.exists(image_dest_path):
            # move images to the respective directory
            shutil.copy(image_path, image_dest_path)

def get_data_loader(batch_size, data_path, train=True):
    print("get data loader")
    if train:
        transform = transforms.Compose([
       transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
         transforms.RandomRotation(10),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
     ])

        image_paths = []
        for class_name in os.listdir(data_path):
            print(class_name)
            class_dir = os.path.join(data_path, class_name)
            for sample_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, sample_name)
                image_paths.append((class_name, image_path))
        # Using Dask Dataframes
        ddf = dd.from_pandas(pd.DataFrame(image_paths, columns=['class', 'path']), npartitions=4)
        # Create a custom dataset instance using your Dask DataFrame
        dataset = CustomDataset(ddf, transform=transform)
        add_augmented_images(data_path, ddf)
        # load data after augmentation
        dataset = datasets.ImageFolder(data_path, transform=transform)

    else:
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        dataset = datasets.ImageFolder(data_path, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=train, num_workers=4, pin_memory=True)
    return data_loader

In [5]:
# Data directories
data_dir = '/kaggle/working/dataset_splits'
train_dir = f'{data_dir}/train'
val_dir = f'{data_dir}/val'
test_dir = f'{data_dir}/test'

In [13]:
# Setup DDP
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("gloo", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)
# Function to save model checkpoints
def save_checkpoint(model, optimizer, epoch, filename="checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.module.state_dict(),  # Note: unwrap the model from DDP
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)
 # Train model using AMP 
def train(model, loader, optimizer, criterion, scaler, rank, epoch, save_interval):
    model.train()
    total_loss = 0.0
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(rank), target.to(rank)
        optimizer.zero_grad()
        with autocast():  # automatic mixed precision
            output = model(data)
            loss = criterion(output, target)
           
           # losses = [torch.tensor(loss).to(rank) for _ in range(2)]
           # print(f"Loss : {losses}")
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        # if rank == 0 and batch_idx % save_interval == 0:
        #     save_checkpoint(model, optimizer, epoch, filename=f"checkpoint_epoch_{epoch}_batch_{batch_idx}.pth")
    return total_loss / len(loader), loss
 
# Validation Function
def validate_model(model, val_loader, rank):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(rank), labels.to(rank)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
 
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')
 
 
# Plot training loss curves
def Training_Loss(epochs, train_loss_gpu1, train_loss_gpu2, train_loss_overall):
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_loss_gpu1, label='GPU 1')
    plt.plot(epochs, train_loss_gpu2, label='GPU 2')
    plt.plot(epochs, train_loss_overall, label='Overall')
    plt.title('Training Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('DDP_Train_Loss_GPUs.png') 
    plt.show()
    
 # Perform training using DDP on multi-GPUs using ResNet50
def ddp_train(rank, world_size, epochs):
    global train_dir, val_dir
    train_loss_gpu1 = []
    train_loss_gpu2 = []
    train_loss_overall = []
    setup(rank, world_size)
    model = resnet50().to(rank)
    model = nn.parallel.DistributedDataParallel(model, device_ids=[rank])
    batch_size = 256
    train_loader = get_data_loader(batch_size // world_size, train_dir, train=True)
    val_loader = get_data_loader(batch_size // world_size, val_dir, train=False)
    criterion = nn.CrossEntropyLoss().to(rank)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    scaler = GradScaler()  # Initialize the gradient scaler for AMP
    save_interval = 100  # Interval for saving checkpoints
 
    for epoch in range(epochs):
        print("Epoch : ", epoch)
        ovr_loss, indi_loss = train(model, train_loader, optimizer, criterion, scaler, rank, epoch, save_interval)
        if rank == 0:
            print(f"Epoch {epoch+1}, Loss: {ovr_loss}")
            save_checkpoint(model, optimizer, epoch, filename=f"checkpoint_epoch_{epoch}.pth")

        # Gather loss from all GPUs
        losses = [torch.tensor(indi_loss).to(rank) for _ in range(world_size)]
        torch.distributed.all_gather(losses, indi_loss)

        # Store loss values from each GPU
        train_loss_gpu1.append(losses[0].item())  # Loss on GPU 1
        train_loss_gpu2.append(losses[1].item())  # Loss on GPU 2

        # Compute overall loss
        train_loss_overall.append(ovr_loss)
        if rank == 0:
            print(f"Epoch {epoch+1}, Loss: {ovr_loss}")
            save_checkpoint(model, optimizer, epoch, filename=f"checkpoint_epoch_{epoch}.pth")
            validate_model(model, val_loader, rank)
            
    # plot training loss on both the GPUs
    epochs = range(1, 11)
    Training_Loss(epochs, train_loss_gpu1, train_loss_gpu2, train_loss_overall)
 
    dist.destroy_process_group()
    

In [ ]:
# Training ResNet 50 pre-trained model on the custom spectrogram data using DDP on 2 GPUs
 
if __name__ == "__main__":
    world_size = 2 
    epochs = 10

    processes = []

    for rank in range(world_size):
        p = mp.Process(target=ddp_train, args=(rank, world_size, epochs))
        p.start()
        processes.append(p)
 
    for p in processes:
        p.join()